In [1]:
#imports
import boto3
import pyodbc
import pandas as pd
import redshift_connector
import psycopg2
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas_gbq
import os
import json
from functools import reduce

In [2]:
with open('aws.json') as json_file:
    cr = json.load(json_file)

s3creds = cr['data_sources']['s3']['creds']
redcreds = cr['data_sources']['redshift']['creds']

In [3]:
#s3 connect 
s3 = boto3.resource(
    service_name='s3',
    region_name=s3creds['region_name'],
    aws_access_key_id=s3creds['aws_access_key_id'],
    aws_secret_access_key=s3creds['aws_secret_access_key']
)

tw_name = 'Tournament_Won.csv'
tl_name = 'Tournament_Lost.csv'

#s3 get data
bucket_name = s3creds['bucket']
tw_file_name = s3creds['folder_path']+tw_name
tl_file_name = s3creds['folder_path']+tl_name
bucket = s3.Bucket(bucket_name)
tw_obj = bucket.Object(key=tw_file_name)
tl_obj = bucket.Object(key=tl_file_name)
tw_response = tw_obj.get()
tl_response = tl_obj.get()
twTable = pd.read_csv(tw_response['Body'],header=3)
tlTable = pd.read_csv(tl_response['Body'],header=3)
twTable.info()
tlTable.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35923 entries, 0 to 35922
Data columns (total 19 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ATP        35923 non-null  int64  
 1   Winner     35923 non-null  object 
 2   WRank      35910 non-null  float64
 3   WPts       35912 non-null  float64
 4   W1         35692 non-null  float64
 5   W2         35353 non-null  float64
 6   W3         16973 non-null  float64
 7   W4         3443 non-null   float64
 8   W5         1295 non-null   float64
 9   Wsets      35693 non-null  float64
 10  B365W      35719 non-null  float64
 11  PSW        32978 non-null  float64
 12  MaxW       29510 non-null  float64
 13  AvgW       29510 non-null  float64
 14  EXW        28712 non-null  float64
 15  LBW        28130 non-null  float64
 16  SJW        15571 non-null  float64
 17  UBW        5309 non-null   float64
 18  row_order  35923 non-null  int64  
dtypes: float64(16), int64(2), object(1)
memory usa

In [4]:
#connecting to redshift using redshift connector

HOST = redcreds['host']
DATABASE = redcreds['database']
USER = redcreds['username']
PASSWORD = redcreds['password']

conn = redshift_connector.connect(
     host=HOST,
     database=DATABASE,
     user=USER,
     password=PASSWORD
  )

print(conn)
cursor = conn.cursor()

query = "select * from tournament_loser"
cursor.execute(query)
tournament_loser : pd.DataFrame = cursor.fetch_dataframe()
print(tournament_loser.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35923 entries, 0 to 35922
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   atp           35923 non-null  int64  
 1   pl2_flag      35338 non-null  object 
 2   pl2_year_pro  35338 non-null  float64
 3   pl2_weight    35338 non-null  float64
 4   pl2_height    35338 non-null  float64
 5   pl2_hand      35338 non-null  object 
 6   row_order     35923 non-null  int64  
dtypes: float64(3), int64(2), object(2)
memory usage: 1.9+ MB
None


In [5]:
#GCP connection
KEYPATH = "gcp.json"
CREDS = service_account.Credentials.from_service_account_file(KEYPATH)
client = bigquery.Client(credentials=CREDS, project=CREDS.project_id)


q1 = """
SELECT * FROM training_dataset.tournament
"""
q2 = """
SELECT * FROM training_dataset.tournament_winner
"""

view1 = client.query(q1)
view2 = client.query(q2)
view1.result()
tournament = view1.result().to_dataframe()
tournament_winner = view2.result().to_dataframe()
tournament.info()
tournament_winner.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35923 entries, 0 to 35922
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ATP              35923 non-null  int64 
 1   Location         35923 non-null  object
 2   Tournament       35923 non-null  object
 3   tournament_date  35923 non-null  object
 4   Series           35923 non-null  object
 5   Court            35923 non-null  object
 6   Surface          35923 non-null  object
 7   Round            35923 non-null  object
 8   best_of          35923 non-null  int64 
 9   Comment          35923 non-null  object
 10  row_order        35923 non-null  int64 
dtypes: int64(3), object(8)
memory usage: 3.0+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35923 entries, 0 to 35922
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ATP           35923 non-null  int64 
 1   pl1_flag      359

In [6]:
dataFrames = [twTable,tlTable,tournament,tournament_winner,tournament_loser]

In [7]:
merged = reduce(lambda left,right: pd.merge(left,right,on='row_order'),dataFrames)

C:\Users\CHANDR~1\AppData\Local\Temp/ipykernel_46064/1287551498.py:1: FutureWarning: Passing 'suffixes' which cause duplicate columns {'ATP_x'} in the result is deprecated and will raise a MergeError in a future version.
  merged = reduce(lambda left,right: pd.merge(left,right,on='row_order'),dataFrames)


In [8]:
mergedStd = merged.drop(['ATP_y','ATP_x','ATP_x','ATP_y'],axis=1)
merged.info()
mergedStd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35923 entries, 0 to 35922
Data columns (total 59 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ATP_x            35923 non-null  int64  
 1   Winner           35923 non-null  object 
 2   WRank            35910 non-null  float64
 3   WPts             35912 non-null  float64
 4   W1               35692 non-null  float64
 5   W2               35353 non-null  float64
 6   W3               16973 non-null  float64
 7   W4               3443 non-null   float64
 8   W5               1295 non-null   float64
 9   Wsets            35693 non-null  float64
 10  B365W            35719 non-null  float64
 11  PSW              32978 non-null  float64
 12  MaxW             29510 non-null  float64
 13  AvgW             29510 non-null  float64
 14  EXW              28712 non-null  float64
 15  LBW              28130 non-null  float64
 16  SJW              15571 non-null  float64
 17  UBW         

In [9]:
pandas_gbq.to_gbq(mergedStd, destination_table = 'training_dataset.DA-12', project_id=CREDS.project_id, credentials = CREDS)